## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Support

### Links: 
[[categories_sc_support]]
___

In [11]:
query_text = """--sql
CREATE TABLE db1.categories_st_support
(
    `report_date` Date,
    `Category` String,
    `Details` String
)
ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/categories_st_support/year=*/month=*/*.csv','CSVWithNames')
SETTINGS format_csv_delimiter = ','
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [13]:
query_text = """--sql
CREATE TABLE db1.categories_st_support_ch
    (
        `report_date` Date,
        `Category` String,
        `Details` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [14]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.categories_st_support_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR TO db1.categories_st_support_ch AS
    SELECT 
        *
    FROM db1.categories_st_support
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [3]:
query_text = """--sql
    SELECT
        *
    FROM db1.categories_st_support
    ORDER BY report_date desc
    limit 10
    """

ch.query_run(query_text)


,report_date,Category,Details
0,2025-09-14,MOB|Info,включить бесключевой доступ
1,2025-09-14,MOB|Access,нет бесключевого доступа
2,2025-09-14,MOB|Info,некорректный адрес в приложении
3,2025-09-14,MOB,приложение не работает
4,2025-09-14,MOB|Auth,проблема с установкой приложения
5,2025-09-14,MOB|Auth,проблема с регистрацией
6,2025-09-14,MOB|Info,дальность бесключевого доступа
7,2025-09-14,MOB|Auth,проблема с авторизацией
8,2025-09-14,MOB|Auth,проблема с приглашением
9,2025-09-14,MOB|Access,код доступа не работает


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.categories_st_support_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [2]:
query_text = """--sql
    DROP TABLE db1.categories_st_support_mv
    """

ch.query_run(query_text)

""


### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.categories_st_support_ch
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [3]:
query_text = """
SYSTEM REFRESH VIEW db1.categories_st_support_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
669573,"""91990ded-8fb1-4c65-8736-4de95c…"
